#, 'trump2020', 'kag2020', 'kag', 'maga2020', 'MAGA','trump', 'americafirst', '', 'wwg1wga', 'NeverBiden','Storm', 'Sleepy joe', 'Raid','CreepyJoeBiden','Riot', 'Terrorists','','Insurrection']
#'trump2020landslide'
#'Trumpsters'

In [ ]:
import snscrape.modules.twitter as sntwitter
import csv
import os
start_date = '2020-11-01'
end_date = '2021-01-07'
tweets_list = []
user_nodes = []
tweetID = []
keyword_count = [0, 0, 0, 0, 0, 0]
keyword = [
            'stoptheseal', 
            'sharpiegate', 
            'riggedelection', 
            'electionfraud', 
            'electionmeddling',
            'qanon'
                ]  

#defintion of the tweet objects
class tweets_det:
    def __init__(self, tweet):
        self.tweetID = tweet.id
        self.tweetDate = tweet.date
        self.userID = tweet.user.id
        self.tweetUserName = tweet.user.username 
        self.tweetContent = tweet.content
        self.likeCount = tweet.likeCount
        self.retweetCount = tweet.retweetCount
        self.follower_id = []      
        self.follower_username = []
        
        self.replycount = tweet.replyCount
        self.quoteCount = tweet.quoteCount
        
class node_user:
    def __init__(self, tweet):
        self.userID = tweet.user.id
        self.username = tweet.user.username
        self.followersCount = tweet.user.followersCount
        self.retweetCount = tweet.retweetCount  
        self.likeCount = tweet.likeCount 
        
        self.replyCount = tweet.replyCount  
        self.quoteCount = tweet.quoteCount  
        self.tweetID = []
        self.keyword = []
        self.node_weight = 0
        self.tweetCount = 1
    def append(tweetID):
        self.tweetID.append(tweet.id)
    def append(follower_id):
        self.follower_id.append(followerID)
    def append(follower_username):
        self.follower_username(follower_username)
    def append(keyword):
        self.keyword(keyword)
        
# The function to read tweets using snscrape and create an object for each tweet
def read_tweets():
    maxTweets = 30000 
    csvFileTweets = open('01_Tweets_List_Latest.csv', 'a', newline='', encoding='utf8')
    csvWriterTweets = csv.writer(csvFileTweets)
    csvWriterTweets.writerow(['Tweet id','Tweet date','UserID', 'User Name', 'tweet Content', 'Reply Count', 'Tweet Retweet count', 'Like Count', 'Quote Count'])         
    
    csvFile_Keyword = open('05_Tweets_Keyword_Count.csv', 'a', newline='', encoding='utf8')
    csvWriter_Keyword = csv.writer(csvFile_Keyword) 
    csvWriter_Keyword.writerow(['Keyword','Weight'])
                               
    for j in range(len(keyword)):
        print (keyword[j]) 
        for i,tweet in enumerate(sntwitter.TwitterSearchScraper(f'{keyword[j]} + since:{start_date} until:{end_date} -filter:links -filter:replies').get_items()):   
            if i > maxTweets:  
                break  
            if (tweet.id not in tweetID):
                csvWriterTweets.writerow([int(tweet.id), tweet.date, tweet.user.id, tweet.user.username, tweet.content, tweet.replyCount, tweet.retweetCount, tweet.likeCount, tweet.quoteCount])
                tweetID.append(tweet.id)
                tweets_list.append(tweets_det(tweet))
                if (len(user_nodes) == 0):
                    user_nodes.append(node_user(tweet))
                    user_nodes[0].tweetID.append(tweet.id)
                    user_nodes[0].keyword.append(keyword)
                    keyword_count[j] = keyword_count[j] + 1
                    user_nodes[0].node_weight += (tweet.retweetCount + tweet.likeCount)
                else:
                    k, found=0, 0  
                    while(k<len(user_nodes) and (found == 0)):
                        if (user_nodes[k].userID==tweet.user.id):
                            found = 1
                            user_nodes[k].replyCount += tweet.replyCount
                            user_nodes[k].retweetCount += tweet.retweetCount
                            user_nodes[k].likeCount += tweet.likeCount
                            user_nodes[k].quoteCount += tweet.quoteCount
                            user_nodes[k].tweetID.append(tweet.id)
                            user_nodes[k].keyword.append(keyword)
                            keyword_count[j] = keyword_count[j] + 1
                            user_nodes[k].node_weight += (tweet.retweetCount + tweet.likeCount)
                            user_nodes[k].tweetCount +=1
                        k+=1
                    if (found ==0 and k == len(user_nodes)):
                        user_nodes.append(node_user(tweet))
                        user_nodes[k].tweetID.append(tweet.id)
                        user_nodes[k].keyword.append(keyword)
                        keyword_count[j] = keyword_count[j] + 1
                        user_nodes[k].node_weight += (tweet.retweetCount + tweet.likeCount)
            else:
                print("Duplicate Tweet found")                    
        csvWriter_Keyword.writerow([keyword[j], int(keyword_count[j])])
    csvFileTweets.close()
    csvFile_Keyword.close()

if __name__=="__main__":  
    read_tweets()
    print("######Reading tweets Done")
    print("######Creating user nodes files in progress")
    csvFile_user_node_graph = open('06_user_node_graph.csv', 'a', newline='', encoding='utf8')
    csvWriter_User = csv.writer(csvFile_user_node_graph) 
    csvWriter_User.writerow(['Label','Node_weight']) 
        
    user_nodes_files = '02_User_Nodes_000_0.csv'
    csvFile9 = open(user_nodes_files, 'a', newline='', encoding='utf8')
    csvWriter9 = csv.writer(csvFile9)
    csvWriter9.writerow(['User ID', 'User Name', 'Node_weight', 'Reply Count', 'Retweet count', 'Like Count', 'Quote Count', 'Followers Count', 'Keywords','Tweet Count']) 

    m = 0
    for i in range (len(user_nodes)):
        csvWriter_User.writerow([user_nodes[i].username, int(user_nodes[i].node_weight)])
        csvWriter9.writerow([int(user_nodes[i].userID), user_nodes[i].username, int(user_nodes[i].node_weight), int(user_nodes[i].replyCount), int(user_nodes[i].retweetCount), int(user_nodes[i].likeCount), int(user_nodes[i].quoteCount), int(user_nodes[i].followersCount), user_nodes[i].keyword, int(user_nodes[i].tweetCount)]) 
        if ((i+1)%1000==0):
            m+=1
            csvFile9.close()
            user_nodes_files = '02_User_Nodes_000_'+str(m)+'.csv'
            print("user_nodes_files: ", user_nodes_files)
            csvFile9 = open(user_nodes_files, 'a', newline='', encoding='utf8')
            csvWriter9 = csv.writer(csvFile9)
            csvWriter9.writerow(['User ID', 'User Name', 'Node_weight', 'Reply Count', 'Retweet count', 'Like Count', 'Quote Count', 'Followers Count', 'Keywords', 'Tweet Count']) 
    csvFile_user_node_graph.close()
    print("User node input file for creating graph- containing username and node_weight created.")
    csvFile9.close()    
    print(len(user_nodes), "#users added to file.")
    print(".....Keyword counts....")
    csvFileKeyword = open('00_Keyword_Count.csv', 'a', newline='', encoding='utf8')
    csvWriterKeywords = csv.writer(csvFileKeyword)
    csvWriterKeywords.writerow(['Keyword', 'Keyword_Count'])
    for x in range(len(keyword)):
        csvWriterKeywords.writerow([keyword[x], int(keyword_count[x])])

In [ ]:
#Convert the user data from file to object while collecting followers data 
#DO NOT FORGET REPLACE YOUR FILE NAME HERE
userfilename = '02_User_Nodes_000_44.csv'
from csv import DictReader
class node_user:
    def __init__(self, user_ID, user_name, node_weight, replyCount, retweetCount, likeCount, quoteCount, followersCount):
        self.userID = user_ID
        self.username = user_name
        self.replyCount = replyCount  
        self.retweetCount = retweetCount  
        self.likeCount = likeCount 
        self.quoteCount = quoteCount  
        self.followersCount = followersCount
        self.follower_id = []  
        self.follower_username = []
        self.tweetID = []
        self.node_weight = 0
    def append(tweetID):
        self.tweetID.append(tweet_id)
    def append(follower_id):
        self.follower_id.append(followerID)
    def append(follower_username):
        self.follower_username(follower_username)
        
if __name__ == '__main__':
    user_nodes_file = open(userfilename , 'r') #
    user_nodes_from = DictReader(user_nodes_file)

    user_nodes = []
    print("Entering for loop")

    for user in user_nodes_from:
        user_ID = user['User ID']      
        user_name = user['User Name']
        replyCount = int(user['Reply Count'])
        retweetCount = int(user['Retweet count'])
        likeCount = int(user['Like Count'])
        quoteCount = int(user['Quote Count'])
        followersCount = int(user['Followers Count'])
        node_weight= int(user['Node_weight'])
        user_nodes.append(node_user(user_ID, user_name, node_weight, replyCount, retweetCount, likeCount, quoteCount, followersCount))
    print("Done")

In [ ]:
#Collecting the followers Data in batches
import csv
import os
from datetime import datetime
from datetime import timezone   
end_date = '2021-01-07'
csvFile3 = open('03_User_Followers_000_01.csv', 'a', newline='', encoding='utf8')
csvWriter3 = csv.writer(csvFile3)
csvFile4 = open('04_User_data_for_graph_01.csv', 'a', newline='', encoding='utf8')
csvWriter4 = csv.writer(csvFile4)
#csvWriter4.writerow(['Source', 'Target', 'Type']) 
#csvWriter3.writerow(['userID', 'username', 'followersCount', 'likeCount', 'retweetCount', 'follower_id_list', 'follower_username_list'])

def strp_time(created_at):
    created_at = datetime.strptime(created_at, '%Y-%m-%dT%H:%M:%S.%fZ').replace(
            tzinfo=timezone.utc).astimezone(tz=None).strftime('%Y-%m-%d')
    return(created_at)

from TwitterAPI import TwitterAPI, TwitterPager, HydrateType

consumer_key        = "TcS7ZvWZjZLkNCMr9vzzjAAwb"
consumer_secret     = "FjXl1Ys2HeL6BqsQpiU813lGRYGDqNOWRekzc9lEmcZX2PvHD5"
access_token        = "1486782696579813378-Qsv8jzODeZXGl7yrjG01PCH6FWhFXL"
access_token_secret = "91vSsGRNWulHQqcQdlqI1M401vVqO4ux6tiPFR0y8TN2Z"

client = TwitterAPI(consumer_key, consumer_secret, access_token, access_token_secret, api_version="2")
followers = []
follower_createdat = []
print("The number of users in the file : ",len(user_nodes))
counter = 0
sleep = 100

for i in range(len(user_nodes)):
#for i in range(54, 55):
    print("User_name : ", user_nodes[i].username)
    params = {
            "max_results": 200,
            "user.fields": "id,username,created_at",
        }
    print("USer_ID in the current loop: ", user_nodes[i].userID)
    pager = TwitterPager(client, f"users/:{user_nodes[i].userID}/followers", params,hydrate_type=HydrateType.APPEND)    
    follower_id = []
    follower_username = []
    for followers in pager.get_iterator(wait=sleep, new_tweets=False):
        counter+=1
        if (strp_time(followers['created_at'])<=end_date):
            follower_id.append(followers['id'])
            follower_username.append(followers['username'])
        if (counter%6000 == 0):
            sleep = sleep+90
    
    for j in range(len(follower_id)):
        user_nodes[i].follower_id.append(follower_id[j])            
        user_nodes[i].follower_username.append(follower_username[j]) 
        csvWriter4.writerow([user_nodes[i].username, follower_username[j], 'Directed']) 
    print(len(follower_id), "no. of followers added to file 4 - graph data- for user- ", user_nodes[i].username)
    csvWriter3.writerow([user_nodes[i].userID, user_nodes[i].username, user_nodes[i].followersCount, user_nodes[i].likeCount, user_nodes[i].retweetCount, follower_id, follower_username])
    print("Data written to file 3")
csvFile3.close()
csvFile4.close() 
print("Done ")
print("Reading followers of all users Done")



In [ ]:
#Collecting the 1000 + 1000 followers of a user 
import csv
import os
from datetime import datetime
from datetime import timezone   
end_date = '2021-01-07'

csvFile4 = open('04_User_data_for_graph_top_10_Part1.csv', 'a', newline='', encoding='utf8')
csvWriter4 = csv.writer(csvFile4)
#csvWriter4.writerow(['Source', 'Target', 'Type']) 
csvFile3 = open('04_User_data_for_graph_top_10_Part2.csv', 'a', newline='', encoding='utf8')
csvWriter3 = csv.writer(csvFile3)
#csvWriter3.writerow(['Source', 'Target', 'Type']) 

def strp_time(created_at):
    created_at = datetime.strptime(created_at, '%Y-%m-%dT%H:%M:%S.%fZ').replace(
            tzinfo=timezone.utc).astimezone(tz=None).strftime('%Y-%m-%d')
    return(created_at)

from TwitterAPI import TwitterAPI, TwitterPager, HydrateType

consumer_key        = "TcS7ZvWZjZLkNCMr9vzzjAAwb"
consumer_secret     = "FjXl1Ys2HeL6BqsQpiU813lGRYGDqNOWRekzc9lEmcZX2PvHD5"
access_token        = "1486782696579813378-Qsv8jzODeZXGl7yrjG01PCH6FWhFXL"
access_token_secret = "91vSsGRNWulHQqcQdlqI1M401vVqO4ux6tiPFR0y8TN2Z"

client = TwitterAPI(consumer_key, consumer_secret, access_token, access_token_secret, api_version="2")
username = 'BernardKerik'
userID = 25837289
follower_createdat = []
counter = 0
sleep = 100

params = {
            "max_results": 200,
            "user.fields": "id,username,created_at",
        }
pager = TwitterPager(client, f"users/:{userID}/followers", params,hydrate_type=HydrateType.APPEND)    
print("Reading followers data")
for followers in pager.get_iterator(wait=sleep, new_tweets=False):
    
    if (strp_time(followers['created_at'])<=end_date):
        counter+=1
        if (counter<=1000):
            csvWriter4.writerow([username, followers['username'], 'Directed'])
        else:
            if(counter<=2000):
                csvWriter3.writerow([username, followers['username'], 'Directed'])
    if (counter==2000):
                break
            
    
    #if (counter%6000 == 0):
        #sleep = sleep+90 

print( "1000 followers added to file 3 and 4 - graph data- for user- ", username)
csvFile4.close() 
csvFile4.close() 
print("Done ")
